#### Imports
<span style="font-family: Arial; font-size: 11pt;">
Der folgende Code enthält die Imports, die für dieses Data Science-Projekt verwendet werden:

- `requests`: Diese Bibliothek ermöglicht das Senden von HTTP-Anfragen an Webseiten und die Verarbeitung der erhaltenen Daten.

- `BeautifulSoup` aus `bs4`: Diese Bibliothek wird verwendet, um HTML- oder XML-Dokumente zu parsen. Sie ermöglicht das Durchsuchen von Webseiten und das Extrahieren spezifischer Daten.

- `psycopg2`: Diese Bibliothek dient als Schnittstelle zur PostgreSQL-Datenbank. Sie ermöglicht das Herstellen von Verbindungen zur Datenbank und das Ausführen von Abfragen sowie andere Datenbankoperationen.</span>

In [3]:
import requests
from bs4 import BeautifulSoup
import psycopg2

##### Connection (URL, DB) / Create Table / Import
<span style="font-family: Arial; font-size: 11pt;">
Der hier von uns geschriebene Code enthält zwei Funktionen, `connect_and_save_data` und `fetch_and_save_duels_stats`, die für das Verbinden mit unserer PostgreSQL-Datenbank, das Abrufen von Daten von einer Webseite und das Speichern dieser Daten in der Datenbank verantwortlich sind.

Die Funktion `connect_and_save_data` verbindet sich mit der PostgreSQL-Datenbank und speichert die übergebenen Daten in der Tabelle `bundesliga_shots_stats`.

Die Funktion `fetch_and_save_shots_stats` durchläuft eine Schleife von 1 bis 34 (Spieltage), ruft Daten von einer Webseite ab, extrahiert relevante Daten und speichert sie in der Datenbank.

Der Code ist spezifisch für das Abrufen und Speichern von Bundesliga-Torschussstatistiken von der Webseite "www.sport.de".
</span>

In [11]:
def connect_and_save_data(matchday, team_name, value):
    try:
        connection = psycopg2.connect(
            dbname='PSQL_ADSFS2023Gruppe15',
            user='ADSFS2023Gruppe15',
            password='ADS_FS_2023_G15!?',
            host='localhost',
            port='5432'
        )

        cursor = connection.cursor()
        
        # Füge die extrahierten Daten in die Tabelle ein
        insert_data_query = '''
            INSERT INTO bundesliga_shots_stats (matchday, team_name, value)
            VALUES (%s, %s, %s);
        '''
        cursor.execute(insert_data_query, (matchday, team_name, value))
        connection.commit()

    except (Exception, psycopg2.Error) as error:
        print('Fehler beim Verbinden mit der PostgreSQL-Datenbank:', error)
    finally:
        # Schließe die Verbindung
        if connection:
            cursor.close()
            connection.close()

# Durchlaufe die Spieltage (md1 bis md34) und lade die Daten für jeden Spieltag
def fetch_and_save_shots_stats():
    for matchday in range(1, 35):
        url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-torschuesse/'
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Suche nach allen Mannschaftsnamen und Torschüssen
            team_elements = soup.find_all('td', class_='team-name team-name-list')
            shots_elements = soup.find_all('td', class_='team_stats-shots team_stats-shots-list')

            # Iteriere über die gefundenen Elemente und extrahiere die gewünschten Informationen
            for team_element, shots_element in zip(team_elements, shots_elements):
                team_name = team_element.text.strip()
                shots_value = shots_element.text.strip()

                # Speichere die Daten in der PostgreSQL-Datenbank
                connect_and_save_data(matchday, team_name, shots_value)

        else:
            print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Torschüsse')


fetch_and_save_shots_stats()

[<td class="team_stats-shots team_stats-shots-list">2</td>, <td class="team_stats-shots team_stats-shots-list">23</td>, <td class="team_stats-shots team_stats-shots-list">19</td>, <td class="team_stats-shots team_stats-shots-list">10</td>, <td class="team_stats-shots team_stats-shots-list">6</td>, <td class="team_stats-shots team_stats-shots-list">18</td>, <td class="team_stats-shots team_stats-shots-list">18</td>, <td class="team_stats-shots team_stats-shots-list">13</td>, <td class="team_stats-shots team_stats-shots-list">13</td>, <td class="team_stats-shots team_stats-shots-list">9</td>, <td class="team_stats-shots team_stats-shots-list">9</td>, <td class="team_stats-shots team_stats-shots-list">30</td>, <td class="team_stats-shots team_stats-shots-list">11</td>, <td class="team_stats-shots team_stats-shots-list">13</td>, <td class="team_stats-shots team_stats-shots-list">16</td>, <td class="team_stats-shots team_stats-shots-list">27</td>, <td class="team_stats-shots team_stats-shot